In [ ]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt

# our code (mark it at autoreload at every cell execution - useful in developement mode)
%load_ext autoreload
%autoreload 1
%aimport Runner_history_utis

In [ ]:
PATH_TO_DATA = '../../Scraping/DataSport/Data/Runners/'
# change the name to the global like website but need to see if the website still works.
RUNNERS_FILE = 'runners_2009.csv'
RUNS_FILE = 'runs_2009.csv'

In [ ]:
runners = pd.read_csv(PATH_TO_DATA + RUNNERS_FILE)
runs = pd.read_csv(PATH_TO_DATA + RUNS_FILE)

In [ ]:
runners.head()

In [ ]:
runs['resultState'].value_counts(dropna=False)

In [ ]:
Runner_history_utis.preprocess_runs(runs)
Runner_history_utis.preprocess_runners(runners)

In [ ]:
runs.head()

In [ ]:
import re

SEMI_MARATHON = '(Semi)|(1/2)|(Halbmarathon)'
MARATHON = 'Marathon'
KILOMETER = '\d*(\.?|\,?)\d*(\s?|-?)[kK][mM]'

OTHER_SPORT = 'Triathlon|Skating|Walking|Duathlon' 

# https://live.escalade.ch/the-race/timetable-prizes-and-courses
GENEVE_RACE = 'Course de l\'Escalade, Genève'
# http://www.christmasmidnightrun.ch/cms/index.php?option=com_content&view=category&layout=blog&id=22&Itemid=107
LAUSSANE_RACE = 'Christmas Midnight Run, Lausanne' 
# http://www.lausanne.ch/course_co
OLYMPIC_RACE = 'Course capitale olympique'
# http://www.sierre-zinal.com/la-course/
SIERRE_ZINAL = 'Sierre-Zinal, Sierre'
# http://www.thyon-dixence.ch/le-parcours/description-2/
THYON_DIXENCE = 'Course pédestre Thyon-Dixence, Thyon'
# https://www.coursedenoel.ch/3222-trail-des-chateaux-2018!.html
SION = 'Course Titzé de noël Sion'
# http://www.morat-fribourg.ch/francais/classique.aspx
MORAT = "Morat-Fribourg"

def compute_distance_from_category (runner):
    attributes = [ 'eventName', 'overCategoryName', 'categoryName']
    
    for attribute in attributes:
        if pd.isnull(runner[attribute]):
            break
            
        if (re.search(OTHER_SPORT, runner[attribute]) != None):
            return None
        
        # check if the string contains information about distance.
        elif (re.search(SEMI_MARATHON, runner[attribute]) != None):
            return 21
        
        elif (re.search(MARATHON, runner[attribute]) != None):
            return 42
        
        elif (re.search(LAUSSANE_RACE, runner[attribute]) != None):
            return 2.4
    
        elif (re.search(GENEVE_RACE, runner[attribute]) != None):
            return 7.3
        
        elif (re.search(OLYMPIC_RACE, runner[attribute]) != None):
            return 5
        
        elif (re.search(SIERRE_ZINAL, runner[attribute]) != None):
            return 31
        
        elif (re.search(THYON_DIXENCE, runner[attribute]) != None):
            return 31
        
        elif (re.search(SION, runner[attribute]) != None):
            return 31
        
        elif (re.search(MORAT, runner[attribute]) != None):
            return 31
        

        # check if the categorie contain km expression
        match = re.search(KILOMETER, runner[attribute])
        # if we match we know the distance.
        if match:
            result = match.group(0).replace(',','.')
            result = re.sub('Km|km|-','',result)
            return round(float(result))
      
    return None

In [ ]:
runs['distance (km)'] = runs.apply(compute_distance_from_category,axis=1)
runs['eventName'][runs['distance (km)'].isnull()].value_counts(dropna=False)